In [8]:
!pip install scikit-learn


     |████████████████████████████████| 11.1 MB 338 kB/s eta 0:00:01
     |████████████████████████████████| 301 kB 57.3 MB/s eta 0:00:01


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import f1_score
import numpy as np
import scipy.sparse as sp
import pandas as pd
import re
train_df = pd.read_csv('filtered_dataset2.csv')
test_df = pd.read_csv('cleaned_dataset1.csv')
train_df.dropna(inplace=True)
def split_entity_value(value):
    match = re.match(r'(\d+\.?\d*)\s*(\w+)', value)  # Matches number followed by a unit
    if match:
        return match.group(1), match.group(2)  # Return number and unit as a tuple
    return None, None  # Return None if no match is found

# Apply the function to the entity_value column
train_df['value'], train_df['unit'] = zip(*train_df['entity_value'].apply(split_entity_value))
# Feature Engineering for train data
train_df.dropna(inplace=True)
train_df['description'] = train_df['description'].fillna('')
test_df['description'] = test_df['description'].fillna('')

# Initialize TfidfVectorizer and LabelEncoder
tfidf = TfidfVectorizer()
label_encoder = LabelEncoder()

# Fit and transform on training data
X_train_description = tfidf.fit_transform(train_df['description'])
X_train_entity_name = label_encoder.fit_transform(train_df['entity_name']).reshape(-1, 1)

# Combine features while keeping X_description in sparse format for train
X_train_entity_name_sparse = sp.csr_matrix(X_train_entity_name)
X_train = sp.hstack((X_train_description, X_train_entity_name_sparse))

# Prepare target variables for train
y_unit_train = train_df['unit']
y_value = train_df['value'].astype(float)
# Prepare test data
X_test_description = tfidf.transform(test_df['description'])
X_test_entity_name = label_encoder.transform(test_df['entity_name']).reshape(-1, 1)
X_test_entity_name_sparse = sp.csr_matrix(X_test_entity_name)
X_test = sp.hstack((X_test_description, X_test_entity_name_sparse))



In [41]:
!pip install xgboost

     |████████████████████████████████| 4.5 MB 8.1 MB/s eta 0:00:01


In [64]:
import xgboost as xgb
model_value =RandomForestRegressor()
model_value.fit(X_train, y_value_train)

RandomForestRegressor()

In [60]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_unit_train)

In [61]:
model_unit = xgb.XGBClassifier(
    objective='multi:softmax',  # Multi-class classification
    num_class=len(label_encoder.classes_),  # Number of unique classes
    use_label_encoder=False,
    eval_metric='mlogloss'
)

model_unit.fit(X_train, y_encoded)


/home/mars/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [21:49:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=26, num_parallel_tree=None, ...)

In [63]:
y_value_pred = model_value.predict(X_test)
test_df['predicted_value'] = y_value_pred
y_unit_pred = model_unit.predict(X_test)
y_unit_pred_original = label_encoder.inverse_transform(y_unit_pred)
test_df['predicted_unit'] =y_unit_pred_original 
# Save the predictions to test.csv
test_df.to_csv('test_out_f.csv', index=True)